In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ImageDatasetLoader") \
    .getOrCreate()


In [2]:
import matplotlib.pyplot as plt
from matplotlib import image
from PIL import Image

In [3]:
import numpy as np

In [4]:
import os


folder_way = os.path.expanduser('home/hduser/Desktop/DATASET')


for dirname, _, filenames in os.walk(folder_way):
    for filename in filenames:
        folder_address = os.path.join(dirname, filename)
        print(folder_address)


In [5]:
os.listdir('/home/hduser/Desktop/DATASET/Decks')

['Cracked', 'Non-cracked']

In [6]:
os.listdir('/home/hduser/Desktop/DATASET/Pavements')

['Cracked', 'Non-cracked']

In [7]:
os.listdir('/home/hduser/Desktop/DATASET/Walls')

['Cracked', 'Non-cracked']

In [8]:
im=image.imread('/home/hduser/Desktop/DATASET/Decks/Cracked/7001-157.jpg')
im.shape

(256, 256, 3)

In [9]:
im=image.imread('/home/hduser/Desktop/DATASET/Pavements/Cracked/001-104.jpg')
im.shape

(256, 256, 3)

In [10]:
 '''
    def load_imgs(impath):
    imgs=[]
    label=[]
    l1=os.listdir(impath)
    for i in l1:
        l2=os.listdir(impath+'/'+i)
        for j in l2:
            img=Image.open(impath+'/'+i+'/'+j)
            img=img.resize(size=(28,28))
            img=np.array(img,dtype=np.float16)/255
            imgs.append(np.array(img))
            label.append(i)
    tg=pd.factorize(np.array(label))
    tr=tg[0]
    return(np.array(imgs),tr)

SyntaxError: incomplete input (1650656119.py, line 1)

In [ ]:
x,y=load_imgs('/home/hduser/Desktop/DATASET/Decks')
print(x, y)

In [11]:
from pyspark.sql import SparkSession
from PIL import Image
import os

# PySpark session'ını başlatın
spark = SparkSession.builder \
    .appName("ImageDatasetLoader") \
    .getOrCreate()

def load_imgs(impath):
    imgs = []
    label = []
    
    # 'impath' yolundaki dosyaları listele
    l1 = os.listdir(impath)
    for i in l1:
        l2 = os.listdir(os.path.join(impath, i))
        for j in l2:
            file_path = os.path.join(impath, i, j)
            
            # Image açma ve boyutlandırma işlemleri
            img = Image.open(file_path)
            img = img.resize(size=(28, 28))
            img_data = np.array(img, dtype=np.float16) / 255
            
            imgs.append(img_data)
            label.append(i)
    
    # Etiketleri faktorize etme işlemi PySpark ile yapılabilir, ancak bu basit örnek için kullanmayacağım.
    # PySpark DataFrames kullanarak daha karmaşık veri işleme yapabilirsiniz.
    
    return np.array(imgs), label

x, y = load_imgs('/home/hduser/Desktop/DATASET/Decks')


In [12]:
print(x, y)

[[[[0.6626 0.6626 0.6704]
   [0.6587 0.6587 0.6665]
   [0.651  0.655  0.6626]
   ...
   [0.6743 0.6743 0.6743]
   [0.6704 0.6704 0.6704]
   [0.686  0.686  0.686 ]]

  [[0.6626 0.6626 0.6704]
   [0.6587 0.6626 0.6704]
   [0.651  0.6587 0.6665]
   ...
   [0.647  0.647  0.647 ]
   [0.6704 0.6704 0.6704]
   [0.6904 0.6904 0.6904]]

  [[0.6665 0.6704 0.678 ]
   [0.6587 0.6665 0.6743]
   [0.643  0.6587 0.6626]
   ...
   [0.651  0.651  0.655 ]
   [0.682  0.682  0.682 ]
   [0.682  0.682  0.682 ]]

  ...

  [[0.678  0.682  0.6904]
   [0.651  0.655  0.6626]
   [0.643  0.647  0.655 ]
   ...
   [0.702  0.706  0.714 ]
   [0.6743 0.678  0.686 ]
   [0.6743 0.678  0.686 ]]

  [[0.682  0.682  0.686 ]
   [0.678  0.678  0.682 ]
   [0.6587 0.6626 0.6704]
   ...
   [0.6943 0.702  0.698 ]
   [0.706  0.71   0.71  ]
   [0.6743 0.678  0.678 ]]

  [[0.6943 0.6943 0.6943]
   [0.6587 0.6587 0.6587]
   [0.6704 0.6743 0.682 ]
   ...
   [0.71   0.718  0.714 ]
   [0.718  0.7256 0.7217]
   [0.702  0.71   0.706 ]]]


 

In [15]:
import numpy as np
from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(np.array(x), np.array(y), test_size=0.2, shuffle=True)


print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


(10896, 28, 28, 3) (2724, 28, 28, 3) (10896,) (2724,)


In [ ]:
from keras.models import Sequential
from keras.layers import MaxPooling2D,Dense,Conv2D,Flatten
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),padding='same',activation='relu',input_shape=x_train.shape[1:]))
model.add(MaxPooling2D())
model.add(Conv2D(64,kernel_size=(3,3),padding='same',activation='relu',input_shape=x_train.shape[1:]))
model.add(MaxPooling2D())
model.add(Conv2D(32,kernel_size=(3,3),padding='same',activation='relu',input_shape=x_train.shape[1:]))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(6, activation='softmax'))
model.summary()